In [52]:
import cv2
import numpy as np

import torch

from PIL import Image

import os,csv
import sys
import numpy as np

# Append custom paths to sys.path for importing custom modules
sys.path.append(os.path.dirname("/home/student/Desktop/31171109-donotdelete/xai-chan/utils"))
from utils.models import ResNet_Model
from utils.transform import resize_transform

In [53]:
MODEL_PATH = "/home/student/Desktop/31171109-donotdelete/XAI-BreakHis/result/results_breakhis_5fold/_Fold_0_5_400X_BreakHis_FT_60_resnet50_MPCS_OP_BS14_epoch500__/_68_91.40625_93.6719786194954_0.9136691689491272.pth"  # TODO: Provide the model path

# Initialize device
device = torch.device('cuda:0')

# Load the model
version = 50
model = ResNet_Model(version=version).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

ResNet_Model(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [21]:
CURRENT_FOLDER = "/home/student/Desktop/31171109-donotdelete/xai-chan/dataset/Fold_3_5/val_10/SOB_M_PC_14-19440/400X"

# List all files in the directory
ALL_FILES = os.listdir(CURRENT_FOLDER)

In [55]:

STATS_PATH = "/home/student/Desktop/31171109-donotdelete/xai-chan/xai"

# Initialize CSV for recording predictions
output_csv_path = os.path.join(STATS_PATH, "mpcs_predict.csv")

# Check if the file exists
file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, 'a') as f:
    writer = csv.writer(f)
    
    # Write the header if the file is new
    if not file_exists:
        writer.writerow(['image_file', 'predicted_category'])

    

    for image_file in ALL_FILES:
        if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image file
            continue
        full_path = os.path.join(CURRENT_FOLDER, image_file)
        print(f"Processing {image_file}...")
        
        input_rgb = Image.open(full_path)
        input_size= input_rgb.size
        input_tensor = resize_transform(np.array(input_rgb)).unsqueeze(0).to(device)
        
        outputs = model(input_tensor)
        target_category = (outputs > 0.2).int().item()
        
        # Write the image filename and its prediction to the CSV
        writer.writerow([image_file, target_category])
    


Processing SOB_B_F-14-21998EF-400-020.png...
Processing SOB_B_A-14-29960CD-400-008.png...
Processing SOB_B_TA-14-15275-400-004.png...
Processing SOB_B_F-14-21998EF-400-014.png...
Processing SOB_B_TA-14-15275-400-005.png...
Processing SOB_M_LC-14-16196-400-016.png...
Processing SOB_M_DC-14-13993-400-024.png...
Processing SOB_M_PC-14-19440-400-015.png...
Processing SOB_M_DC-14-13993-400-007.png...
Processing SOB_B_A-14-29960CD-400-009.png...
Processing SOB_B_PT-14-29315EF-400-006.png...
Processing SOB_B_A-14-29960CD-400-007.png...
Processing SOB_M_PC-14-19440-400-025.png...
Processing SOB_B_TA-14-15275-400-010.png...
Processing SOB_M_PC-14-19440-400-020.png...
Processing SOB_M_MC-14-13418DE-400-008.png...
Processing SOB_B_PT-14-29315EF-400-015.png...
Processing SOB_M_LC-14-16196-400-012.png...
Processing SOB_M_PC-14-19440-400-011.png...
Processing SOB_M_DC-14-13993-400-001.png...
Processing SOB_B_A-14-29960CD-400-016.png...
Processing SOB_M_MC-14-13418DE-400-005.png...
Processing SOB_M_L

LRP 

In [ ]:
for image_file in ALL_FILES:
        full_path = os.path.join(CURRENT_FOLDER, image_file)
        print(f"Processing {image_file}...")
        
        input_rgb = Image.open(full_path)
        input_size= input_rgb.size
        input_tensor = resize_transform(np.array(input_rgb)).unsqueeze(0).to(device)
        
        outputs = model(input_tensor)
        target_category = (outputs > 0.2).int().item()

In [23]:
from pytorch_grad_cam import GradCAM, ScoreCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [57]:
def generate_gradcam_heatmap(image_path, model, target_layer, targets):
    input_rgb = Image.open(image_path)
    input_size= input_rgb.size
    input_tensor = resize_transform(np.array(input_rgb)).unsqueeze(0).to(device)
    # Convert to NumPy array if it's not already
    if not isinstance(input_rgb, np.ndarray):
        input_rgb = np.array(input_rgb)

    # Normalize to [0, 1]
    input_rgb = input_rgb.astype(np.float32) / 255.0

    cam = GradCAM(model=model, target_layers=target_layer, use_cuda=True)
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    grayscale_cam_resized = cv2.resize(grayscale_cam, (input_rgb.shape[1], input_rgb.shape[0]))
    visualization = show_cam_on_image(np.array(input_rgb), grayscale_cam_resized, use_rgb=True)

    return visualization

CURRENT_FOLDER = "/home/student/Desktop/31171109-donotdelete/xai-chan/xai/400X-images"
ALL_FILES = os.listdir(CURRENT_FOLDER)
targets = [ClassifierOutputTarget(0)]
# Define the target layer
target_layer = [model.model.layer4[-1]]
print(target_layer)
for image_file in ALL_FILES:
    if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image file
        continue

    full_path = os.path.join(CURRENT_FOLDER, image_file)
    print(f"Processing {image_file}...")
    
    heatmap = generate_gradcam_heatmap(full_path, model, target_layer, targets)
    
    # Save the heatmap
    output_path = os.path.join(CURRENT_FOLDER, "mpcs", f"heatmap_{image_file}")
    Image.fromarray(heatmap).save(output_path)

[Bottleneck(
  (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
)]
Processing SOB_B_F-14-21998EF-400-020.png...
Processing SOB_B_A-14-29960CD-400-008.png...
Processing SOB_B_TA-14-15275-400-004.png...
Processing SOB_B_F-14-21998EF-400-014.png...
Processing SOB_B_TA-14-15275-400-005.png...
Processing SOB_M_LC-14-16196-400-016.png...
Processing SOB_M_DC-14-13993-400-024.png...
Processing SOB_M_PC-14-19440-400-015.png...
Processing SOB_M_DC-14-13993-400-007.png...
Processing SOB_B_A-14-29960CD-400-009.png.

In [62]:
import os
from zennit.image import imgify, imsave
from zennit.torchvision import ResNetCanonizer
from zennit.composites import EpsilonPlusFlat, EpsilonAlpha2Beta1, EpsilonPlus,EpsilonAlpha2Beta1Flat
from zennit.attribution import Gradient
from PIL import Image
import torch
def generate_lrp_heatmap(image_path, model, device):
    """
    Generate LRP heatmap for a given image.
    
    Parameters:
    - image_path (str): Path to the image file.
    - model (torch.nn.Module): The PyTorch model to be used.
    - resize_transform (torchvision.transforms.Compose): Transform for resizing and normalizing the image.
    - device (torch.device): Device (CPU or CUDA).
    
    Returns:
    - heatmap_resized (PIL.Image.Image): The resulting LRP visualization.
    """
    input_rgb = Image.open(image_path)
    input_size= input_rgb.size
    # print(f"Width: {width}, Height: {height}")
    input_tensor = resize_transform(np.array(input_rgb)).unsqueeze(0).to(device)
    
    
    
    # Get model prediction
    outputs = model(input_tensor)
    target_category = (outputs > 0.2).int().item()
    
    # Define canonizer and composite for LRP
    canonizer = ResNetCanonizer()
    composite = EpsilonPlusFlat(canonizers=[canonizer])
    target = torch.tensor([[target_category]]).float().to(device)
    
    # Calculate LRP attribution
    with Gradient(model=model, composite=composite) as attributor:
        output, attribution = attributor(input_tensor, target)
    
    # Convert attribution to visualization
    relevance = attribution.sum(1).cpu()
    heatmap = imgify(relevance, symmetric=True, cmap="coldnhot")
    heatmap_resized = heatmap.resize(input_size)
    
    
    return heatmap_resized

CURRENT_FOLDER = "/home/student/Desktop/31171109-donotdelete/xai-chan/xai/400X-images"
ALL_FILES = os.listdir(CURRENT_FOLDER)

for image_file in ALL_FILES:
    if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image file
        continue

    full_path = os.path.join(CURRENT_FOLDER, image_file)
    print(f"Processing {image_file}...")
    
    heatmap = generate_lrp_heatmap(full_path, model, device)
    
    # Save the heatmap
    output_path = os.path.join(CURRENT_FOLDER, "mpcs",f"EpsilonPlusFlat{image_file}")
    imsave(output_path, heatmap, format='PNG',cmap='coldnhot')

Processing SOB_B_F-14-21998EF-400-020.png...
Processing SOB_B_A-14-29960CD-400-008.png...
Processing SOB_B_TA-14-15275-400-004.png...
Processing SOB_B_F-14-21998EF-400-014.png...
Processing SOB_B_TA-14-15275-400-005.png...
Processing SOB_M_LC-14-16196-400-016.png...
Processing SOB_M_DC-14-13993-400-024.png...
Processing SOB_M_PC-14-19440-400-015.png...
Processing SOB_M_DC-14-13993-400-007.png...
Processing SOB_B_A-14-29960CD-400-009.png...
Processing SOB_B_PT-14-29315EF-400-006.png...
Processing SOB_B_A-14-29960CD-400-007.png...
Processing SOB_M_PC-14-19440-400-025.png...
Processing SOB_B_TA-14-15275-400-010.png...
Processing SOB_M_PC-14-19440-400-020.png...
Processing SOB_M_MC-14-13418DE-400-008.png...
Processing SOB_B_PT-14-29315EF-400-015.png...
Processing SOB_M_LC-14-16196-400-012.png...
Processing SOB_M_PC-14-19440-400-011.png...
Processing SOB_M_DC-14-13993-400-001.png...
Processing SOB_B_A-14-29960CD-400-016.png...
Processing SOB_M_MC-14-13418DE-400-005.png...
Processing SOB_M_L